In [1]:
import cirq
import numpy as np

from cirq_sic import *

This modular addition algorithm works by temporarily calculating values like a+k. Since a and k can both be as large as d-1, their sum can reach up to (d-1) + (d-1) = 2d-2.

Your qubit register can only represent integers from 0 to $2^n-1$. For the algorithm's logic (especially the part that checks for overflow using the most significant bit) to be reliable, the temporary sum must not exceed the register's capacity.

This leads to the requirement: $2d-2 < 2^n$. Thus we can encode a qudit in $n$ qubits with $d$ satisfying

$$d \leq 2^{n-1}  $$

https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/quantum_modular_artihmetics.ipynb

In [ ]:
def Z_d(d, qubits, aux, k=1):
    """Z acting on the first d basis vectors of n qubits. Requires two auxilliary qubits. Note d <= 2^{n-1}"""
    extended_qubits = [aux[0]] + qubits
    yield from Z(extended_qubits, k=k)
    yield from Z(extended_qubits, k=-d)
    yield from qft(extended_qubits, inverse=True)
    yield cirq.CNOT(extended_qubits[0], aux[1])
    yield from qft(extended_qubits)
    yield from [op.controlled_by(aux[1]) for op in Z(extended_qubits, k=d)]
    yield from Z(extended_qubits, k=-k)
    yield from qft(extended_qubits, inverse=True)
    yield cirq.X(extended_qubits[0])
    yield cirq.CNOT(extended_qubits[0], aux[1])
    yield cirq.X(extended_qubits[0])
    yield from qft(extended_qubits)
    yield from Z(extended_qubits, k=k)

def X_d(d, qubits, aux, k=1):
    """X acting on the first d basis vectors of n qubits. Requires two auxilliary qubits. Note d <= 2^{n-1}"""
    extended_qubits = [aux[0]] + qubits
    yield from qft(extended_qubits)
    yield from Z_d(d, qubits, aux, k=k)
    yield from qft(extended_qubits, inverse=True)

In [ ]:
def CZ_d(d, control_qubits, target_qubits, aux, inverse=False):
    """CZ acting on the first d basis vectors of two pairs of n qubits. Requires two auxilliary qubits. Note d <= 2^{n-1}"""
    if inverse:
        yield from cirq.inverse(list(CZ_d(d, control_qubits, target_qubits, aux)))
        return
    for i, control_qubit in enumerate(control_qubits):
        k = 2**(len(control_qubits) - 1 - i)
        yield from [op.controlled_by(control_qubit) for op in Z_d(d, target_qubits, aux, k=k)]

def CX_d(d, control_qubits, target_qubits, aux, inverse=False):
    """CX acting on the first d basis vectors of two pairs of n qubits. Requires two auxilliary qubits. Note d <= 2^{n-1}"""
    extended_target_qubits = [aux[0]] + target_qubits
    yield from qft(extended_target_qubits)
    yield from CZ_d(d, control_qubits, target_qubits, aux, inverse=inverse)
    yield from qft(extended_target_qubits, inverse=True)

In [ ]:
def qft_d(d, qubits, inverse=False):
    """QFT acting on the first d basis vectors of two pairs of n qubits."""
    if inverse:
        yield from cirq.inverse(qft_d(d, qubits))
        return
    n = len(qubits)
    F = np.array([[np.exp(2*np.pi*1j*i*j/d) for j in range(d)] for i in range(d)])/np.sqrt(d)
    Fd_gate = cirq.MatrixGate(sc.linalg.block_diag(F, np.eye(2**n - d)), name=f"DFT({d})")
    yield from cirq.decompose(cirq.Circuit((Fd_gate.on(*qubits))))

In [59]:
def mod_d_outcome_mask(d, n, m):
    """When working on computations mod d encoded in n-qubits, with m groups of n-qubits."""
    d_b = 2**n
    return sum([kron(*[np.eye(d_b, dtype=int)[i] for i in ind]) for ind in np.ndindex(*[d_b]*m) if np.all(np.array(ind) < d)])

def mod_d_probabilities(p, d, n, m):
    return p[np.where(mod_d_outcome_mask(d, n, m)==1)]

In [57]:
n = 2
d = 2**n
phi = rand_ket(d)
ansatz_preparation = ansatz_circuit(phi)

system_qubits = cirq.LineQubit.range(n)
ancilla_qubits = cirq.LineQubit.range(n, 2*n)
aux = [cirq.NamedQubit("aux1"), cirq.NamedQubit("aux2")]
sim = cirq.Simulator()

circ1 = cirq.Circuit((ansatz_preparation(system_qubits, conjugate=False),
                      X(system_qubits)))
circ2 = cirq.Circuit((ansatz_preparation(system_qubits, conjugate=False),
                      X_d(d, system_qubits, aux, k=1)))

res1, res2 = sim.simulate(circ1),sim.simulate(circ2)
rho1, rho2= res1.density_matrix_of(system_qubits), res2.density_matrix_of(system_qubits)
assert np.allclose(rho1, rho2)

circ1 = cirq.Circuit((ansatz_preparation(ancilla_qubits, conjugate=True),
                      ansatz_preparation(system_qubits, conjugate=False),
                      CX(ancilla_qubits, system_qubits)))
circ2 = cirq.Circuit((ansatz_preparation(ancilla_qubits, conjugate=True),
                      ansatz_preparation(system_qubits, conjugate=False),
                      CX_d(d, ancilla_qubits, system_qubits, aux)))
res1, res2 = sim.simulate(circ1), sim.simulate(circ2)
rho1, rho2 = res1.density_matrix_of(ancilla_qubits+system_qubits),\
             res2.density_matrix_of(ancilla_qubits+system_qubits)
assert np.allclose(rho1, rho2)

In [60]:
n = 3
d = 2**n
d_s = 3
phi = load_sic_fiducial(d_s)
ket = rand_ket(d_s)
embedded_phi = np.concatenate([phi, np.zeros(d-d_s)])
embedded_ket = np.concatenate([ket, np.zeros(d-d_s)])
prepare_fiducial = ansatz_circuit(embedded_phi)
prepare_system = ansatz_circuit(embedded_ket)

system_qubits = cirq.LineQubit.range(n)
ancilla_qubits = cirq.LineQubit.range(n, 2*n)
aux = [cirq.NamedQubit("aux1"), cirq.NamedQubit("aux2")]
sim = cirq.Simulator()

circ = cirq.Circuit((prepare_system(system_qubits, conjugate=False),
                     prepare_fiducial(ancilla_qubits, conjugate=True),
                     CX_d(d_s, ancilla_qubits, system_qubits, aux, inverse=True),
                     qft_d(d_s, ancilla_qubits, inverse=True)))
res = sim.simulate(circ)
total_p = np.diag(res.density_matrix_of(system_qubits+ancilla_qubits)).real
p = mod_d_probabilities(total_p, d_s, n, 2)

E = wh_povm(phi)
p2 = change_conjugate_convention(np.array([ket.conj() @ e @ ket for e in E]).real)
assert np.allclose(p, p2)